In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('train_aprel/train.csv')

In [3]:
df_train.head()

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [4]:
df_train.label.unique()

array([9, 0, 3, 2, 7, 5, 1, 6, 4, 8])

In [5]:
from keras.preprocessing import image

Using TensorFlow backend.


In [6]:
from tqdm import tqdm

In [7]:
df_train['id'] = df_train['id'].apply(str)

In [8]:
import numpy as np
train_image = []
for i in tqdm(range(df_train.shape[0])):
  img = image.load_img('train_aprel/train/' + df_train['id'][i] + '.png' , target_size = (28,28,1),grayscale = True)
  img = image.img_to_array(img)
  img = img/255
  train_image.append(img)
X = np.array(train_image)

  0%|          | 0/60000 [00:00<?, ?it/s]/home/harish/my_project_dir/my_project_env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 60000/60000 [14:50<00:00, 67.34it/s] 


In [9]:
res = X.tolist()

In [10]:
df_train['id'] = res

In [11]:
df_train.head(1)

,id,label
0,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",9


In [12]:
from keras.utils import to_categorical

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
y=df_train['label'].values
y = to_categorical(y)

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))




In [28]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 24s 497us/step - loss: 0.7992 - accuracy: 0.7080 - val_loss: 0.5086 - val_accuracy: 0.8138
Epoch 2/20
48000/48000 [==============================] - 22s 462us/step - loss: 0.5498 - accuracy: 0.8037 - val_loss: 0.4426 - val_accuracy: 0.8354
Epoch 3/20
48000/48000 [==============================] - 21s 448us/step - loss: 0.4790 - accuracy: 0.8274 - val_loss: 0.3918 - val_accuracy: 0.8558
Epoch 4/20
48000/48000 [==============================] - 21s 448us/step - loss: 0.4378 - accuracy: 0.8427 - val_loss: 0.3946 - val_accuracy: 0.8549
Epoch 5/20
48000/48000 [==============================] - 21s 447us/step - loss: 0.4103 - accuracy: 0.8533 - val_loss: 0.3446 - val_accuracy: 0.8702
Epoch 6/20
48000/48000 [==============================] - 22s 449us/step - loss: 0.3912 - accuracy: 0.8606 - val_loss: 0.3421 - val_accuracy: 0.8752
Epoch 7/20
48000/48000 [==============================] 

In [30]:
pred = model.predict_classes(X_test)

In [31]:
pred

array([8, 8, 7, ..., 8, 8, 1])

In [32]:
df_test = pd.read_csv('test_aprel/test.csv')

In [33]:
df_test.dtypes

id    int64
dtype: object

In [34]:
df_test["id"]= df_test["id"].astype(str)

In [35]:
test_image = []
for i in tqdm(range(df_test.shape[0])):
  img  = image.load_img('test_aprel/test/' + df_test['id'][i] + ".png", target_size = (28,28,1),grayscale=True)
  img = image.img_to_array(img)
  img = img/255
  test_image.append(img)

X1 = np.array(test_image)

  0%|          | 0/10000 [00:00<?, ?it/s]/home/harish/my_project_dir/my_project_env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 10000/10000 [01:27<00:00, 114.92it/s]


In [36]:
pred = model.predict_classes(X1)

In [37]:
prediction = pred.tolist()

In [38]:
df_sub = pd.read_csv('sample_aprel.csv')

In [39]:
df_sub['label'] = prediction

In [40]:
df_sub

,id,label
0,60001,9
1,60002,2
2,60003,1
3,60004,1
4,60005,6
...,...,...
9995,69996,9
9996,69997,1
9997,69998,8
9998,69999,1


In [41]:
df_sub.to_csv('aprel_result_f.csv',index = False)